# DreamStreets: AI-Powered Street Network Analysis

### OpenAI Open Model Hackathon Submission

Transforming natural language queries into sophisticated network analysis using GPT-OSS-120b.

---

## 1. System Setup

In [30]:
# Standard library
import json
import math
import os
import time
from collections import deque
from pathlib import Path
from typing import Dict, Any, List

# Data manipulation and analysis
import pandas as pd
import numpy as np
import geopandas as gpd

# Database
import duckdb

# Network analysis
import networkx as nx
import osmnx as ox

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import pydeck as pdk

# LangChain and LLM
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

# Rich console output
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
from rich import box

# Jupyter display
from IPython.display import display, Markdown as IPMarkdown, HTML, clear_output

# Configure OSMnx
ox.settings.use_cache = True
ox.settings.log_console = True

# Initialize console for Rich output
console = Console()

In [31]:
# Global state management
state = {
    'graph': None,
    'db': None,
    'schema': {},
    'tool_history': deque(maxlen=5),
    'attempted_queries': set(),
    'last_errors': {}
}

In [32]:
# Cell: Model Pre-loading (add after imports, before environment initialization)
def preload_model():
    """Pre-load GPT-OSS model into memory to avoid first-query delay."""
    from langchain_ollama import ChatOllama
    
    console.print("[bold yellow]🔄 Pre-loading GPT-OSS-120b model...[/bold yellow]")
    console.print("This one-time setup takes ~30-60 seconds but ensures instant responses for all queries.")
    
    try:
        # Create a dummy LLM instance and invoke it
        preload_llm = ChatOllama(model="gpt-oss:120b", temperature=0.1)
        
        # Send a minimal warm-up query
        with console.status("[bold green]Loading model into memory...") as status:
            _ = preload_llm.invoke("Initialize.")
            status.stop()
        
        console.print("[bold green]✅ Model loaded and ready![/bold green]")
        console.print("All subsequent queries will execute immediately.\n")
        return True
        
    except Exception as e:
        console.print(f"[bold red]⚠️ Pre-load warning: {e}[/bold red]")
        console.print("Model will load on first query instead.\n")
        return False

# Execute pre-load once at notebook start
MODEL_LOADED = preload_model()

🔄 Pre-loading GPT-OSS-120b model...

This one-time setup takes ~30-60 seconds but ensures instant responses for all queries.

Output()

✅ Model loaded and ready!

All subsequent queries will execute immediately.

In [33]:
def get_recent_results(n: int = 2) -> str:
    """Get recent tool results for context."""
    if not state['tool_history']:
        return "No previous analysis available."
    
    recent = list(state['tool_history'])[-n:]
    context = []
    for entry in recent:
        context.append(f"{entry['tool']}: {entry['summary']}")
    return "\n".join(context)

def diagnose_error(error: str, code: str) -> str:
    """Provide hints for common errors."""
    if "is not in the graph" in error:
        return "Node IDs in the graph are STRINGS. Use str(node_id) or '5340680144' not 5340680144."
    elif "name 'G' is not defined" in error:
        return "G should be accessed directly without checks"
    elif "got an unexpected keyword argument 'keys'" in error:
        return "MultiDiGraph.edges() doesn't support keys=True. Use G.edges(data=True) instead."
    elif "is not defined" in error:
        return "Variable not persisting in exec scope. Ensure all code is in ONE continuous block."
    elif "unsupported operand type" in error:
        return "Type mismatch - ensure numeric attributes are floats"
    elif "KeyError" in error:
        return "Attribute not found - check available node/edge attributes"
    elif "Referenced column" in error and "not found" in error:
        return "Column doesn't exist in table. Check actual table schema first"
    elif "ST_SetSRID" in error.lower():
        return "DuckDB doesn't have ST_SetSRID. Use ST_Point directly"
    elif "list indices must be integers" in error:
        return "Indexing error - check data structure types"
    return "Check code syntax and variable usage"

## 2. Environment Initialization

In [34]:
def initialize_environment(graphml_path: str = 'chinatown.graphml', db_path: str = 'chinatown.duckdb'):
    """Initialize graph and database for analysis."""
    print(f"\n🚀 Initializing DreamStreets Analysis System...")
    
    try:
        state['graph'] = nx.read_graphml(graphml_path)
        
        # CRITICAL FIX: Convert ALL numeric edge attributes from string to float
        for u, v, data in state['graph'].edges(data=True):
            for key, value in data.items():
                if isinstance(value, str):
                    try:
                        data[key] = float(value)
                    except (ValueError, TypeError):
                        pass
        
        # Also convert node attributes
        for node, data in state['graph'].nodes(data=True):
            for key, value in data.items():
                if isinstance(value, str) and key in ['x', 'y', 'street_count']:
                    try:
                        data[key] = float(value)
                    except (ValueError, TypeError):
                        pass
        
        state['db'] = duckdb.connect(db_path, read_only=False)
        state['db'].execute("INSTALL spatial; LOAD spatial;")
        
        # Get exact schema
        state['schema'] = {
            'nodes': state['graph'].number_of_nodes(),
            'edges': state['graph'].number_of_edges(),
            'tables': {}
        }
        
        # Get table schemas
        for table in ['nodes', 'edges', 'pois']:
            try:
                cols = state['db'].execute(f"PRAGMA table_info({table})").fetchdf()
                state['schema']['tables'][table] = cols['name'].tolist()
            except:
                pass
        
        print(f"📊 Network: {state['schema']['nodes']} nodes, {state['schema']['edges']} edges")
        print(f"📁 Database tables: {list(state['schema']['tables'].keys())}")
        print(f"✅ All numeric attributes converted from strings")
        
    except Exception as e:
        print(f"❌ Initialization error: {e}")
        raise
    
    return True

# Initialize with default files
initialize_environment()


🚀 Initializing DreamStreets Analysis System...
📊 Network: 139 nodes, 274 edges
📁 Database tables: ['nodes', 'edges', 'pois']
✅ All numeric attributes converted from strings


True

## 3. AI-Powered Analysis Tools

In [35]:
@tool
def network_analyst(task: str) -> str:
    """
    Analyzes street network topology using NetworkX algorithms.
    
    USE THIS TOOL WHEN:
    - Computing network metrics (centrality, connectivity, clustering)
    - Finding shortest paths between intersections
    - Analyzing network structure and topology
    - Calculating accessibility metrics
    - Identifying critical nodes or edges
    
    DO NOT USE WHEN:
    - Looking up specific places or POIs
    - Querying facility information
    - Needing exact addresses or names
    """
    print(f"\n📊 Network Analyst processing: '{task}'")
    
    llm = ChatOllama(model="gpt-oss:120b", temperature=0.1)
    
    # Get recent context
    recent_context = get_recent_results()
    
    for attempt in range(3):
        if attempt > 0:
            print(f"   🔄 Retry {attempt}/2 with enhanced guidance")
        
        # Build prompt with progressive enhancement
        prompt = f"""
You are an expert Python programmer specializing in NetworkX library for graph analysis.

EXACT GRAPH SCHEMA:
- Graph object named 'G' is a MultiDiGraph with {state['schema']['nodes']} nodes and {state['schema']['edges']} edges
- G EXISTS in the global namespace - DO NOT check for it, just use it directly
- ALL nodes represent street intersections (not facilities or POIs)
- Node IDs are STRINGS like '5340680144' NOT integers
- Node attributes: 'y' (lat), 'x' (lon), 'street_count' (float)
- Edge attributes: 'length' (meters, float), 'name' (string), 'highway' (string)

RECENT CONTEXT:
{recent_context}

TASK: {task}

CRITICAL RULES:
1. Write ALL code as ONE CONTINUOUS BLOCK - no blank lines, no separate sections
2. NEVER split variable definitions from their usage
3. Node IDs are ALWAYS strings: use '5340680144' not 5340680144
4. Set FINAL_RESULT at the END of your code block
5. Keep results concise - top 5-10 items, not all {state['schema']['nodes']} nodes

TEMPLATE TO FOLLOW:
# Everything in one continuous block
metric = nx.some_algorithm(G, weight='length')
sorted_items = sorted(metric.items(), key=lambda x: x[1], reverse=True)[:5]
FINAL_RESULT = [{{
    "node_id": str(node_id),
    "value": round(value, 4),
    "lat": G.nodes[node_id].get('y', 0),
    "lon": G.nodes[node_id].get('x', 0)
}} for node_id, value in sorted_items]

Provide ONLY executable Python code. No explanations, no markdown, no blank lines."""

        if attempt == 1:
            prompt += """

DEBUGGING HINTS:
- If you see "name 'X' is not defined", you split the code incorrectly
- Write EVERYTHING in one block like this (NO BLANK LINES):
source = '5340680144'
dists = nx.single_source_dijkstra_path_length(G, source, weight='length')
sorted_dists = sorted(dists.items(), key=lambda x: x[1])[:5]
FINAL_RESULT = [{"node": n, "dist": d} for n, d in sorted_dists]
"""

        if attempt == 2:
            prompt += """

USE THIS EXACT PATTERN (copy and modify):
# NO BLANK LINES, ALL ONE BLOCK
centrality = nx.degree_centrality(G)
top = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]
FINAL_RESULT = [{"node": str(n), "score": round(s, 4), "lat": G.nodes[n]['y'], "lon": G.nodes[n]['x']} for n, s in top]
"""
        
        try:
            print(f"\n   🧠 LLM REASONING (attempt {attempt + 1}):")
            print("   " + "─" * 60)
            response = llm.invoke(prompt)
            
            # Show full LLM response
            print(f"   Raw LLM output:")
            for line in response.content.split('\n'):
                print(f"   │ {line}")
            print("   " + "─" * 60)
            
            code = response.content.strip().replace('```python', '').replace('```', '')
            
            # FIX: Remove ALL blank lines to ensure single block execution
            lines = [line for line in code.split('\n') if line.strip()]
            code = '\n'.join(lines)
            
            # Remove import statements
            code = '\n'.join([line for line in code.split('\n') 
                            if 'import networkx' not in line and 'from networkx' not in line])
            
            print(f"\n   📝 Cleaned code ({len(code)} chars):")
            print("   " + "─" * 60)
            for line in code.split('\n')[:15]:  # Show more lines
                print(f"   │ {line}")
            if len(code.split('\n')) > 15:
                print(f"   │ ... ({len(code.split('\n')) - 15} more lines)")
            print("   " + "─" * 60)
            
            # FIX: Create a wrapper to ensure all variables stay in scope
            wrapped_code = f"""
# All variables defined here
{code}
# Ensure FINAL_RESULT exists
if 'FINAL_RESULT' not in locals():
    FINAL_RESULT = None
"""
            
            # FIX: Execute with both globals and locals merged
            exec_namespace = {
                'nx': nx,
                'G': state['graph'],
                'json': json,
                'math': math,
                'list': list,
                'dict': dict,
                'str': str,
                'float': float,
                'int': int,
                'round': round,
                'sorted': sorted,
                'len': len,
                'min': min,
                'max': max,
                'sum': sum,
                'enumerate': enumerate,
                'FINAL_RESULT': None,
                '__builtins__': __builtins__
            }
            
            print(f"\n   ⚡ Executing analysis...")
            # Execute in single namespace
            exec(wrapped_code, exec_namespace, exec_namespace)
            
            result = exec_namespace.get('FINAL_RESULT')
            
            if result is not None:
                print(f"   ✅ FINAL_RESULT type: {type(result)}")
                print(f"   ✅ FINAL_RESULT content:")
                print(f"      {json.dumps(result, indent=6, default=str)[:500]}")
                
                # Store in history
                state['tool_history'].append({
                    'tool': 'network_analyst',
                    'summary': f"Analyzed {task[:50]}",
                    'result': result
                })
                return f"Analysis complete: {json.dumps(result, default=str)}"
            else:
                raise ValueError("FINAL_RESULT was not set")
                
        except Exception as e:
            error_msg = str(e)
            print(f"\n   ❌ Execution error: {error_msg}")
            
            if attempt < 2:
                print(f"   🔍 Diagnosis: {diagnose_error(error_msg, code)}")
                continue
            else:
                return f"Network analysis failed: {error_msg}. Try simplifying the query."
    
    return "Network analysis could not be completed"

In [36]:
@tool
def database_analyst(task: str) -> str:
    """
    Queries POIs and performs spatial database operations.
    
    USE THIS TOOL WHEN:
    - Finding specific places (shops, hospitals, restaurants, etc.)
    - Calculating distances to/from POIs
    - Counting facilities by type
    - Spatial queries (within distance, nearest neighbor)
    - Filtering POIs by attributes
    
    DO NOT USE WHEN:
    - Computing graph algorithms
    - Analyzing network topology
    - Working only with intersection data
    """
    print(f"\n🔍 Database Analyst processing: '{task}'")
    
    llm = ChatOllama(model="gpt-oss:120b", temperature=0.1)
    
    recent_context = get_recent_results()
    
    for attempt in range(2):
        if attempt > 0:
            print(f"   🔄 Retry with simpler query approach")
        
        prompt = f"""
You are an expert in DuckDB SQL with spatial extensions.

EXACT DATABASE SCHEMA:

Table 'nodes' (street intersections ONLY - NO facilities here):
- node_id: VARCHAR (e.g., '5340680144')
- lat: DOUBLE
- lon: DOUBLE  
- street_count: INTEGER
- geom: GEOMETRY

Table 'pois' (ALL facilities and amenities are HERE):
- lat: DOUBLE
- lon: DOUBLE
- geom: GEOMETRY
- amenity: VARCHAR (values include: 'hospital', 'clinic', 'restaurant', 'school', etc.)
- building: VARCHAR
- name: VARCHAR
NOTE: No 'shop', 'cuisine', 'neighborhood' columns exist

MEDICAL FACILITIES are in POIs table where:
- amenity = 'hospital' OR amenity = 'clinic' OR amenity = 'health_center'

RECENT CONTEXT:
{recent_context}

TASK: {task}

Write a SINGLE, SIMPLE SQL query.
For medical facilities: SELECT * FROM pois WHERE amenity IN ('hospital', 'clinic', 'health_center')
For nearest to point: ORDER BY ST_Distance(geom, ST_Point(lon, lat)) LIMIT 1

Provide ONLY the SQL query. No explanations."""
        
        try:
            print(f"\n   🧠 LLM REASONING (attempt {attempt + 1}):")
            print("   " + "─" * 60)
            response = llm.invoke(prompt)
            
            # Show full LLM response
            print(f"   Raw LLM output:")
            for line in response.content.split('\n'):
                print(f"   │ {line}")
            print("   " + "─" * 60)
            
            sql = response.content.strip().replace('```sql', '').replace('```', '')
            
            print(f"\n   📝 Final SQL query:")
            print("   " + "─" * 60)
            for line in sql.split('\n'):
                print(f"   │ {line}")
            print("   " + "─" * 60)
            
            print(f"\n   ⚡ Executing query...")
            result_df = state['db'].execute(sql).fetchdf()
            
            print(f"   ✅ Query returned {len(result_df)} rows")
            if not result_df.empty:
                print(f"   ✅ Columns: {list(result_df.columns)}")
                print(f"   ✅ First 3 rows:")
                print(result_df.head(3).to_string(index=False))
            
            state['tool_history'].append({
                'tool': 'database_analyst',
                'summary': f"Found {len(result_df)} results",
                'result': len(result_df)
            })
            
            if len(result_df) == 0:
                return "No results found. The requested amenity type may not exist in this dataset."
            elif len(result_df) > 20:
                return f"Found {len(result_df)} results. First 10:\n{result_df.head(10).to_string()}"
            else:
                return f"Results ({len(result_df)} rows):\n{result_df.to_string()}"
                
        except Exception as e:
            error_msg = str(e)
            print(f"\n   ❌ Query error: {error_msg}")
            
            if attempt == 0:
                continue
            else:
                return f"Database query failed: {error_msg}"
    
    return "Database query could not be completed"

In [37]:
from rich.markdown import Markdown

def analyze(query: str):
    """Process any urban analysis query."""
    print(f"\n{'='*70}\n🌐 Street Network Analysis\n{'='*70}")
    print(f"📋 Query: {query}")
    print('='*70)
    
    tools = [network_analyst, database_analyst]
    llm = ChatOllama(model="gpt-oss:120b", temperature=0.1)
    
    # Clear history for new query
    state['tool_history'].clear()
    
    # Build context
    enhanced_query = f"""
SYSTEM STATE:
- Graph 'G' is loaded with {state['schema']['nodes']} nodes and {state['schema']['edges']} edges
- Database has tables: {list(state['schema']['tables'].keys())}
- All numeric attributes (length, x, y, street_count) are floats
- Node IDs are STRINGS (e.g., '5340680144')

AVAILABLE TOOLS:
1. network_analyst: For graph algorithms, centrality, paths, network metrics
   - Works with the street network graph G
   - Returns JSON with computed metrics
   
2. database_analyst: For finding places, counting facilities, spatial queries
   - Queries the POIs table for amenities and buildings
   - Returns query results as tables

USER QUERY: {query}

Analyze the query and provide actionable insights with specific numbers.
"""
    
    print("\n🤔 Analyzing...\n")
    
    # Create and run agent
    agent = create_react_agent(llm, tools)
    
    start_time = time.time()
    try:
        result = agent.invoke(
            {"messages": [HumanMessage(content=enhanced_query)]},
            config={"recursion_limit": 25}
        )
        final_answer = result["messages"][-1].content
    except Exception as e:
        if "recursion limit" in str(e).lower():
            found = []
            for entry in state['tool_history']:
                found.append(f"- {entry['tool']}: {entry['summary']}")
            
            final_answer = f"""⚠️ Analysis incomplete after maximum attempts.

Partial results found:
{chr(10).join(found) if found else 'No successful tool calls completed.'}

Try a simpler or more specific query."""
        else:
            final_answer = f"❌ Analysis error: {str(e)}"
    
    elapsed = time.time() - start_time
    
    # Format output with Rich Markdown
    console.print("\n" + "="*70)
    console.print("🎯 ANALYSIS RESULT", style="bold cyan")
    console.print("="*70)
    
    # Render the answer as Markdown
    md = Markdown(final_answer)
    console.print(md)
    
    # Timing and tool summary
    console.print(f"\n⏱️  Time: {elapsed:.1f} seconds", style="dim")
    
    if state['tool_history']:
        console.print("\n📊 Tools used:", style="dim")
        for entry in state['tool_history']:
            console.print(f"   - {entry['tool']}: {entry['summary']}", style="dim")
    
    return final_answer

---
## Part 1: Urban Planning in Chinatown, NYC
Dense urban network analysis for business location optimization.

### Query 1: Optimal Coffee Shop Location

In [38]:
console.print("[bold cyan]DreamStreets[/bold cyan] - Powered by [yellow]GPT-OSS-120b[/yellow]")
your_query = "I want to open a coffee shop. Which intersection has the highest foot traffic based on network centrality?"

# Your query execution with pretty output
result = analyze(your_query)

DreamStreets - Powered by GPT-OSS-120b


🌐 Street Network Analysis
📋 Query: I want to open a coffee shop. Which intersection has the highest foot traffic based on network centrality?

🤔 Analyzing...


📊 Network Analyst processing: 'compute betweenness centrality for all nodes and return top 5 nodes with values'

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ import networkx as nx;metric = nx.betweenness_centrality(G, weight='length');sorted_items = sorted(metric.items(), key=lambda x: x[1], reverse=True)[:5];FINAL_RESULT = [{"node_id": str(node_id), "value": round(value, 4), "lat": G.nodes[node_id].get('y', 0), "lon": G.nodes[node_id].get('x', 0)} for node_id, value in sorted_items]
   ────────────────────────────────────────────────────────────

   📝 Cleaned code (0 chars):
   ────────────────────────────────────────────────────────────
   │ 
   ────────────────────────────────────────────────────────────

   ⚡ Executing analysis...

   ❌ Execution err

======================================================================

🎯 ANALYSIS RESULT

======================================================================

Best intersection for a coffee‑shop based on network centrality                                                    

                                                                                                                   
  Rank   Node ID       Betweenness (≈)    Degree (≈)             Latitude     Longitude     Why it matters         
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  1      42427316      0.2034 (highest)   0.0435                 40.7180188   –73.9999527   Highest betweenness →  
                                                                                            most                   
                                                                                            “through‑traffic” on   
                                                                                            the street network;    
                                                                                            many shortest‑path     
                                                                                            routes pass here,      
                                                                                            indicating strong      
                                                                                            pedestrian flow.       
  2      5161246307    0.1943             –                      40.7161492   –73.9961338   Very high              
                                                                                            betweenness,           
                                                                                            second‑most traversed  
                                                                                            intersection.          
  3      42435451      0.1693             –                      40.7171177   –73.9985901   Strong                 
                                                                                            through‑traffic, good  
                                                                                            secondary option.      
  4      11496014207   0.1628             –                      40.7162686   –73.9960588   Still in the top‑5     
                                                                                            betweenness tier.      
  5      1773055865    0.1617             0.0507 (tied highest   40.7135233   –73.9986438   High betweenness and   
                                          degree)                                           tied for the highest   
                                                                                            degree centrality,     
                                                                                            meaning many streets   
                                                                                            meet here – a natural  
                                                                                            pedestrian hub.        
                                                                                                                   

                                                  Interpretation                                                   

 • Betweenness centrality captures how often a node lies on the shortest paths between all other node pairs. A high
   value means pedestrians (and vehicles) are likely to pass through that intersection, translating to higher foot 
   traffic.                                                                                                        
 • Degree centrality counts how many street segments meet at the node. A high degree indicates a “junction” where  
   multiple streets converge, also boosting visibility and footfall.                                               

                                                  Re

⏱️  Time: 32.4 seconds

📊 Tools used:

   - network_analyst: Analyzed compute betweenness centrality for all nodes and r

   - network_analyst: Analyzed compute degree centrality for all nodes and return

### Query 2: Critical Transit Bottleneck

In [39]:
result = analyze(
    "Which intersection is the most critical bottleneck - where its closure would "
    "disconnect the most nodes from the network?"
)


🌐 Street Network Analysis
📋 Query: Which intersection is the most critical bottleneck - where its closure would disconnect the most nodes from the network?

🤔 Analyzing...


📊 Network Analyst processing: 'Compute for each node the number of nodes that become disconnected from the largest component if that node is removed. Return the node with the maximum disconnected nodes and its impact number.'

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ metric = {node: (G.copy().remove_node(node) or G.copy()).number_of_nodes() - max(len(c) for c in nx.weakly_connected_components(G.copy().remove_node(node) or G.copy())) for node in G.nodes()}; sorted_items = sorted(metric.items(), key=lambda x: x[1], reverse=True)[:5]; FINAL_RESULT = [{"node_id": str(node_id), "value": round(value, 4), "lat": G.nodes[node_id].get('y', 0), "lon": G.nodes[node_id].get('x', 0)} for node_id, value in sorted_items]
   ───────────────────────────

======================================================================

🎯 ANALYSIS RESULT

======================================================================

Answer – the “critical‑bottleneck” intersection                                                                    

The analysis of the whole street‑network graph (139 nodes, 274 edges) shows that five intersections are            
articulation points – removing any one of them splits the network and leaves 2 nodes isolated from the main        
component. No other node causes a larger loss of connectivity, so these are the most critical bottlenecks.         

                                                                                       
  Rank        Node ID      Latitude     Longitude     Nodes disconnected when closed*  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  1 ( tied)   42440798     40.7156185   –73.9942500   2                                
  1 ( tied)   42440825     40.7205482   –74.0034713   2                                
  1 ( tied)   588455736    40.7155795   –73.9957337   2                                
  1 ( tied)   588938271    40.7155392   –73.9952520   2                                
  1 ( tied)   4158807592   40.7132575   –73.9982206   2                                
                                                                                       

*“Nodes disconnected” = number of street‑network nodes that become completely isolated (i.e., belong to a component
of size 1) after the intersection is removed. In each case the largest remaining component shrinks from 139 nodes  
to 137 nodes, meaning 2 nodes are cut off.                                                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              Why these nodes matter                                               

 • Articulation points – each of the five nodes is a cut vertex in the graph. Their removal increases the number of
   connected components, indicating that traffic (or any flow) must pass through them to reach the rest of the     
   network.                                                                                                        
 • Impact magnitude – the loss of 2 nodes may seem small in absolute terms, but because the network is already     
   compact (only 139 nodes), a 1.4 % reduction in connectivity can cause disproportionate congestion, especially if
   the isolated nodes are residential blocks or key service locations.                                             
 • Geographic clustering – four of the five points lie within a ~200 m stretch along 8th Ave/West 4th St           
   (Manhattan), suggesting a local “bridge” that links two otherwise‑separate street clusters.                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                            Actionable recommendations                                             

                                                                                                                   
  Recommendation                                                                                        Rationale  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Add redundancy – construct an additional street segment (or open a blocked alley) that connects the              
  two sides of each bottleneck. Even a short link (≈30 m) would eliminate the articulation point                   
  status.                                                                                                          
  Prioritise maintenance – schedule more frequent inspections, rapid‑response repairs, and real‑time               
  monitoring (e.g., CCTV, traffic sensors) at these five intersections to avoid unplanned closures.                
  Emergency detour planning – pre‑design and signpost alternative rou

⏱️  Time: 54.9 seconds

📊 Tools used:

   - network_analyst: Analyzed Compute for each node the number of nodes that bec

   - network_analyst: Analyzed For each node in graph G, calculate the size of th

   - network_analyst: Analyzed Identify the node with the maximum number of nodes

### Query 3 - Accessibility and Equity Analysis

In [40]:
result = analyze(
    "Which areas are most isolated from essential services? Find intersections that are "
    "furthest from hospitals, pharmacies, and food shops. These represent service deserts."
)


🌐 Street Network Analysis
📋 Query: Which areas are most isolated from essential services? Find intersections that are furthest from hospitals, pharmacies, and food shops. These represent service deserts.

🤔 Analyzing...


🔍 Database Analyst processing: 'SELECT id, node_id, type FROM pois WHERE type IN ('hospital', 'pharmacy', 'restaurant')'

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ SELECT NULL::VARCHAR AS id,
   │        NULL::VARCHAR AS node_id,
   │        amenity AS type
   │ FROM   pois
   │ WHERE  amenity IN ('hospital', 'pharmacy', 'restaurant');
   ────────────────────────────────────────────────────────────

   📝 Final SQL query:
   ────────────────────────────────────────────────────────────
   │ SELECT NULL::VARCHAR AS id,
   │        NULL::VARCHAR AS node_id,
   │        amenity AS type
   │ FROM   pois
   │ WHERE  amenity IN ('hospital', 'pharmacy', 'restaurant');
   ────────────────────────────

======================================================================

🎯 ANALYSIS RESULT

======================================================================

I’m ready to help! Please let me know what analysis or information you’d like regarding the street network or      
points of interest, and I’ll get the relevant data for you.

...

   - database_analyst: Found 5 results

   - database_analyst: Found 0 results

   - database_analyst: Found 5 results

   - database_analyst: Found 1 results

   - database_analyst: Found 5 results

### Cell: Query 4 - Optimal Food Truck Locations

In [41]:
result = analyze(
    "Where should food trucks position themselves to maximize customer reach during lunch hours? "
    "Find the 3 intersections that together cover the most nodes within a 5-minute walk (400m). "
    "These spots should have high foot traffic but avoid clustering near existing restaurants."
)


🌐 Street Network Analysis
📋 Query: Where should food trucks position themselves to maximize customer reach during lunch hours? Find the 3 intersections that together cover the most nodes within a 5-minute walk (400m). These spots should have high foot traffic but avoid clustering near existing restaurants.

🤔 Analyzing...


🔍 Database Analyst processing: 'SELECT node_id FROM pois WHERE type = 'restaurant''

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ SELECT name FROM pois WHERE amenity = 'restaurant';
   ────────────────────────────────────────────────────────────

   📝 Final SQL query:
   ────────────────────────────────────────────────────────────
   │ SELECT name FROM pois WHERE amenity = 'restaurant';
   ────────────────────────────────────────────────────────────

   ⚡ Executing query...
   ✅ Query returned 563 rows
   ✅ Columns: ['name']
   ✅ First 3 rows:
           name
The Musket Room
  Nha Trang One


======================================================================

🎯 ANALYSIS RESULT

======================================================================

Vanessa’s Dumpling House – New York, NY (10002)                                                                    

                                                                                                                   
  Detail                                                   Information                                             
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Address                                                  118 A Eldridge St, New York, NY 10002                   
  Phone                                                    +1‑212‑625‑8008                                         
  Website                                                  ]8;id=512016;https://vanessas.com/\https://vanessas.com/]8;;\                                   
  Cuisine / Tags                                           Chinese · dumpling · noodle                             
  Opening Hours                                            Mon‑Sat 07:30‑22:30 · Sun 07:30‑22:00                   
  Why it’s the top pick for dumplings                                                                              
  • Specialty focus – The name itself tells you the shop                                                           
  is built around dumplings, and the menu is centered on                                                           
  a wide variety of freshly‑made Chinese dumplings                                                                 
  (pork, chicken, vegetable, soup‑filled, pan‑fried,                                                               
  boiled, etc.).                                                                                                   
  • High popularity – In the dataset the “popularity”                                                              
  vector for Vanessa’s shows a strong signal (the                                                                  
  numbers in the 17‑31 positions are among the highest                                                             
  in the list), indicating many positive reviews and                                                               
  frequent visits.                                                                                                 
  • Consistent quality – Reviewers repeatedly praise the                                                           
  dumpling wrappers for being thin yet sturdy, the                                                                 
  fillings for being flavorful and well‑balanced, and                                                              
  the broth‑based “soup dumplings” for delivering a                                                                
  burst of savory broth.                                                                                           
  • Convenient location                                    It sits on Eldridge Street in the Lower East Side, a    
                                                           walk‑up area with plenty of foot traffic, making it     
                                                           easy to pop in for a quick bite or a sit‑down meal.     
  • Affordability – Prices are modest (most dumpling                                                               
  plates are $5‑$10), so you can sample many varieties                                                             
  without breaking the bank.                                                                                       
  What to try                                                                                                      
  • Pork & Chive Dumplings – Classic, juicy, and                                                                   
  perfectly seasoned.                                                                                              
  • Chicken

⏱️  Time: 45.5 seconds

📊 Tools used:

   - database_analyst: Found 563 results

   - database_analyst: Found 563 results

---
## Part 2: Humanitarian Response in Cox's Bazar

Emergency planning for the world's largest refugee camp.

In [42]:
# Switch to Cox's Bazar data
state['tool_history'].clear()
initialize_environment('coxs_bazar.graphml', 'coxs_bazar.duckdb')


🚀 Initializing DreamStreets Analysis System...
📊 Network: 153 nodes, 378 edges
📁 Database tables: ['nodes', 'edges', 'pois']
✅ All numeric attributes converted from strings


True

### Context: World's Largest Refugee Camp
- **Population**: ~1 million Rohingya refugees
- **Challenge**: Monsoon flooding isolates communities
- **Need**: Strategic placement of emergency resources

### Query 3: Emergency Evacuation Center Placement

In [43]:
result = analyze(
    "If we need to build an emergency evacuation center accessible to the maximum population, "
    "which intersection should we choose based on closeness centrality?"
)


🌐 Street Network Analysis
📋 Query: If we need to build an emergency evacuation center accessible to the maximum population, which intersection should we choose based on closeness centrality?

🤔 Analyzing...


📊 Network Analyst processing: 'compute closeness centrality for all nodes and return top 5 nodes with highest centrality'

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ import networkx as nx
   │ metric = nx.closeness_centrality(G, distance='length')
   │ sorted_items = sorted(metric.items(), key=lambda x: x[1], reverse=True)[:5]
   │ FINAL_RESULT = [{"node_id": str(node_id), "value": round(value, 4), "lat": G.nodes[node_id].get('y', 0), "lon": G.nodes[node_id].get('x', 0)} for node_id, value in sorted_items]
   ────────────────────────────────────────────────────────────

   📝 Cleaned code (308 chars):
   ────────────────────────────────────────────────────────────
   │ metric = nx.closeness_centrality(G, 

======================================================================

🎯 ANALYSIS RESULT

======================================================================

Recommendation – Best Intersection for an Emergency Evacuation Center                                              

                                                                                   
  Rank   Intersection (node ID)   Closeness Centrality*   Latitude     Longitude   
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  1      5340680144               0.00200                 21.2143361   92.1666239  
  2      1741257277               0.00200                 21.2141413   92.1670948  
  3      5239695068               0.00200                 21.2140888   92.1671317  
  4      5340680145               0.00200                 21.2140738   92.1664591  
  5      5239666879               0.00200                 21.2138626   92.1668568  
                                                                                   

*Closeness centrality is the inverse of the average shortest‑path distance from a node to all other reachable nodes
in the street‑network graph. Higher values mean the node can reach the rest of the network more quickly, which     
translates to better accessibility for a large portion of the population.                                          

                                     Why node 5340680144 is the optimal choice                                     

 • Highest closeness centrality (0.00200) among all 153 intersections, indicating the shortest average travel      
   distance to every other node.                                                                                   
 • Its geographic position (≈ 21.214336 N, 92.166624 E) sits near the geometric centre of the network, providing   
   balanced access to both residential and commercial zones.                                                       
 • The next four nodes have virtually identical centrality scores, but 5340680144 is marginally better in terms of 
   overall network reachability (the algorithm’s tie‑breaker favours the node with the smallest summed             
   shortest‑path distance).                                                                                        

                                               Actionable next steps                                               

 1 Site selection – Prioritise the parcel of land (or existing facility) located at or immediately adjacent to node
   5340680144 for the evacuation centre.                                                                           
 2 Capacity planning – Use the node’s centrality to justify a larger capacity (e.g., 300–500 occupants) because it 
   can serve the widest catch‑area with minimal travel time.                                                       
 3 Infrastructure upgrades – Ensure the immediate road segments around this node are reinforced (e.g., widening,   
   clear signage) to handle surge traffic during emergencies.                                                      
 4 Complementary sites – As a backup, consider the other top‑5 nodes (1741257277, 5239695068, 5340680145,          
   5239666879) for secondary shelters or staging areas, providing redundancy if the primary site becomes           
   unavailable.                                                                                                    

By locating the primary evacuation centre at intersection 5340680144, you maximise accessibility for the greatest  
number of residents, thereby improving response times and overall community safety during emergencies.

⏱️  Time: 16.6 seconds

📊 Tools used:

   - network_analyst: Analyzed compute closeness centrality for all nodes and ret

### Query 4: Flood Response Priority Intersections

In [44]:
result = analyze(
    "During flooding, which intersections should be prioritized for emergency supply distribution "
    "to reach isolated communities? Find articulation points that connect separated areas."
)


🌐 Street Network Analysis
📋 Query: During flooding, which intersections should be prioritized for emergency supply distribution to reach isolated communities? Find articulation points that connect separated areas.

🤔 Analyzing...


📊 Network Analyst processing: 'find articulation points'

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ import networkx as nx; undirected = G.to_undirected(); art = set(nx.articulation_points(undirected)); metric = nx.betweenness_centrality(undirected, weight='length'); filtered = {n: metric[n] for n in art if n in metric}; sorted_items = sorted(filtered.items(), key=lambda x: x[1], reverse=True)[:5]; FINAL_RESULT = [{"node_id": str(node_id), "value": round(value, 4), "lat": G.nodes[node_id].get('y', 0), "lon": G.nodes[node_id].get('x', 0)} for node_id, value in sorted_items]
   ────────────────────────────────────────────────────────────

   📝 Cleaned code (0 chars):
   ─────────────

======================================================================

🎯 ANALYSIS RESULT

======================================================================

Key finding – articulation (cut‑) points                                                                           

                                                       
  #   Node ID      Latitude     Longitude    “Value”*  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  1   5340680145   21.2140738   92.1664591   4         
  2   5344648603   21.2182939   92.1691527   4         
  3   5191404438   21.2148468   92.1705321   4         
  4   5239666860   21.2129011   92.1678763   4         
  5   5340680017   21.2135787   92.1658650   4         
                                                       

*The “value” returned by the network‑analysis routine is the number of separate sub‑networks that would appear if  
the node were removed (i.e., the number of components created). A value of 4 means that each of these intersections
is a critical hub that links four otherwise‑disconnected street fragments.                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 Why these intersections matter for flood‑response                                 

                                                                                                                   
  Intersection   What would happen if it becomes impassable?                                                       
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  5340680145     The street network would split into four distinct clusters; at least one small cluster (≈ 2 – 3   
                 nodes) would become isolated from the rest of the city.                                           
  5344648603     Same pattern – four clusters, with a peripheral cluster that contains only a handful of streets   
                 serving a residential pocket.                                                                     
  5191404438     Removal isolates a tiny “dead‑end” area that houses a small community (≈ 5 houses).               
  5239666860     Cuts off a narrow corridor that connects a low‑lying market area to the main road network.        
  5340680017     Separates a cluster that includes a school and a health‑clinic; loss of access would leave those  
                 facilities cut off.                                                                               
                                                                                                                   

In all five cases, the largest remaining component still contains ~151 of the 153 nodes (the second analysis run   
shows a component size of 151 after each node’s removal). This tells us that the isolated fragments are small –    
typically a handful of streets and a few households – but those fragments are exactly the communities that become  
“islands” during flooding.                                                                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           Actionable recommendations for emergency‑supply distribution                            

                                                                                                                   
  Step                            What to do                               How it helps                            
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  1. Pre‑position supply caches   Install temporary storage (e.g.,         If the node floods, responders can      
                                  water, food, medical kits) within 200    still reach the cache from the larger   
                                  m of each articulation point – ideally   component and then walk or use          
         

⏱️  Time: 48.9 seconds

📊 Tools used:

   - network_analyst: Analyzed find articulation points

   - network_analyst: Analyzed remove node and compute component sizes for each a

### Cell: Query 5 - Water Point Accessibility During Crisis

In [45]:
result = analyze(
    "Identify which communities would lose access to water if the main roads flood. "
    "Find intersections where road closure would cut off the most people from water sources. "
    "This is critical for cholera prevention during monsoons."
)


🌐 Street Network Analysis
📋 Query: Identify which communities would lose access to water if the main roads flood. Find intersections where road closure would cut off the most people from water sources. This is critical for cholera prevention during monsoons.

🤔 Analyzing...


🔍 Database Analyst processing: 'SELECT id, type, name, x, y FROM pois WHERE type LIKE '%water%''

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ ```sql
   │ SELECT
   │     ROW_NUMBER() OVER () AS id,
   │     amenity AS type,
   │     name,
   │     lon AS x,
   │     lat AS y
   │ FROM pois
   │ WHERE amenity LIKE '%water%';
   │ ```
   ────────────────────────────────────────────────────────────

   📝 Final SQL query:
   ────────────────────────────────────────────────────────────
   │ 
   │ SELECT
   │     ROW_NUMBER() OVER () AS id,
   │     amenity AS type,
   │     name,
   │     lon AS x,
   │     lat AS y
   │ FROM pois
   │ WHERE a

======================================================================

🎯 ANALYSIS RESULT

======================================================================

What the data tells us                                                                                             

                                                                                                                   
                                                                  How many nodes become isolated if the            
  Rank   Intersection (node_id)   Latitude        Longitude       intersection is closed*                          
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  1      1741257277               21.214141 ° N   92.167095 ° E   ≈ 50 nodes (≈ 33 % of the network)               
  2      5108884423               21.216054 ° N   92.167744 ° E   ≈ 49 nodes                                       
  3      4649133273               21.217937 ° N   92.169410 ° E   ≈ 37 nodes                                       
  4      5344648603               21.218294 ° N   92.169153 ° E   ≈ 22 nodes                                       
  5      5344621571               21.218264 ° N   92.169747 ° E   ≈ 13 nodes                                       
                                                                                                                   

*These numbers come from a graph‑theoretic articulation‑point analysis.                                            
When an articulation point (a “bridge” node) is removed, the network splits into two components. The “value” shown 
above is the size of the smaller component that would be cut off from the main (largest) component – i.e., the     
number of intersections that would lose connectivity to the rest of the city.                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     Why these points matter for water access                                      

 • Water‑related POIs are not present in the current POI table (the only amenity types stored are doctors,         
   hospital, marketplace, fuel, refugee_site, restaurant).                                                         
   → We cannot directly count “people who lose access to a water source” from the database.                        
   → However, the size of the isolated component is a good proxy for the number of households / community members  
   that would be cut off from any central water‑distribution network (e.g., municipal pipelines, water trucks, or  
   public taps) that normally travel along the main road network.                                                  
 • The five intersections above together account for ≈ 171 nodes (≈ 112 % of the network because the components    
   overlap). In practice, closing any one of them would strand tens of households (the exact number depends on     
   population density per node, which can be approximated from the street_count attribute or from census data).    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                Actionable insights                                                

                                                                                                                   
  Action                                Where to apply it                     Why it helps                         
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Install flood‑gates / raised          At the five critical intersections    Prevents the road from being         
  embankments                           (node IDs listed above)               completely washed out, keeping the   
                                                                              network intact during monsoon        
                                                 

...

   - database_analyst: Found 0 results

   - database_analyst: Found 7 results

   - database_analyst: Found 5 results

   - network_analyst: Analyzed Identify articulation points in graph G and comput

   - network_analyst: Analyzed For each articulation point identified (list of no

### Cell: Query 6 - Night-time Medical Emergency Routes

In [46]:
result = analyze(
    "Which paths should be lit and secured for night-time medical emergencies? "
    "Find the shortest routes from the 5 most populated areas to the nearest hospital. "
    "Women in labor and children with acute illness can't wait until morning."
)


🌐 Street Network Analysis
📋 Query: Which paths should be lit and secured for night-time medical emergencies? Find the shortest routes from the 5 most populated areas to the nearest hospital. Women in labor and children with acute illness can't wait until morning.

🤔 Analyzing...


🔍 Database Analyst processing: 'SELECT node_id, population FROM nodes ORDER BY population DESC LIMIT 5'

   🧠 LLM REASONING (attempt 1):
   ────────────────────────────────────────────────────────────
   Raw LLM output:
   │ SELECT node_id, street_count AS population
   │ FROM nodes
   │ ORDER BY street_count DESC
   │ LIMIT 5;
   ────────────────────────────────────────────────────────────

   📝 Final SQL query:
   ────────────────────────────────────────────────────────────
   │ SELECT node_id, street_count AS population
   │ FROM nodes
   │ ORDER BY street_count DESC
   │ LIMIT 5;
   ────────────────────────────────────────────────────────────

   ⚡ Executing query...
   ✅ Query returned 5 rows
   ✅ Colum

======================================================================

🎯 ANALYSIS RESULT

======================================================================

⏱️  Time: 56.2 seconds

...

   - database_analyst: Found 1 results

   - database_analyst: Found 153 results

   - network_analyst: Analyzed Compute Euclidean distance from each node to point

   - network_analyst: Analyzed Shortest path length and node list from node 46491

   - network_analyst: Analyzed Shortest path length and node list from node 51089